# Instant Gratification!

Let's build a useful LLM solution - in a matter of minutes.

Our goal is to code a new kind of Web Browser. Give it a URL, and it will respond with a summary. The Reader's Digest of the internet!!

Before starting, be sure to have followed the instructions in the "README" file, including creating your API key with OpenAI and adding it to the `.env` file.

In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# Connecting to OpenAI

The next cell is where we load in the environment variables in your `.env` file and connect to OpenAI.

Troubleshooting if you have problems:

1. OpenAI takes a few minutes to register after you set up an account. If you receive an error about being over quota, try waiting a few minutes and try again.
2. As a fallback, replace the line `openai = OpenAI()` with `openai = OpenAI(api_key="your-key-here")` - while it's not recommended to hard code tokens in Jupyter lab, because then you can't share your lab with others, it's a workaround for now
3. Contact me! Message me or email ed@edwarddonner.com and we will get this to work.

Any concerns about API costs? See my notes in the README - costs should be minimal, and you can control it at every point.

In [2]:
# Load environment variables in a file called .env

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
openai = OpenAI()

In [3]:
# A class to represent a Webpage

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [4]:
# Let's try one out

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connect
with me for

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [5]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [6]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "The contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

In [7]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

## Time to bring it together - the API for OpenAI is very simple!

In [10]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [12]:
summarize("https://edwarddonner.com")

"# Summary of Edward Donner's Website\n\nEdward Donner's website serves as a platform for sharing insights on artificial intelligence, specifically focusing on large language models (LLMs). He expresses a passion for coding, DJing, and electronic music production. Edward is the co-founder and CTO of Nebula.io, which utilizes AI to enhance talent discovery and management processes. He previously founded the AI startup untapt, which was acquired in 2021.\n\n## Recent Posts\n- **Outsmart LLM Arena** (August 6, 2024): An announcement about an interactive platform where LLMs compete in strategy and negotiation.\n- **Choosing the Right LLM: Toolkit and Resources** (June 26, 2024): A guide for selecting suitable LLMs for various applications.\n- **Fine-tuning an LLM on Your Texts: A Simulation of You** (February 7, 2024): Discussion on personalizing LLMs for individual use.\n- **Fine-tuning an LLM on Your Texts: Part 4 – QLoRA** (January 31, 2024): Detailed exploration of a specific method fo

In [13]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [14]:
display_summary("https://edwarddonner.com")

# Summary of Edward Donner's Website

The website serves as a personal and professional platform for Ed Donner, a programmer, LLM (Large Language Model) enthusiast, and the co-founder and CTO of Nebula.io. The site includes insights into Ed's interests in coding, electronic music production, and his background in AI startups.

## Key Highlights:

- **Outsmart**: An initiative that features LLMs competing in scenarios that require diplomacy and cunning.
- **About Ed**: Ed shares his professional journey, including the founding of Nebula.io and his previous startup, untapt, which was acquired in 2021. He emphasizes using AI to help people fulfill their potential.
- **Posts and Resources**: The site includes several posts discussing topics such as selecting the right LLM, fine-tuning LLMs, and practical tools for working with them.

## Recent Announcements:
1. **August 6, 2024**: Announcement of the "Outsmart LLM Arena".
2. **June 26, 2024**: Introduction of resources for selecting the right LLM.
3. **February 7, 2024**: Post on fine-tuning LLMs to simulate personal text styles.
4. **January 31, 2024**: Continuation of the fine-tuning series focusing on QLoRA.

Overall, the website is a blend of personal exploration in AI, professional endeavors, and educational content centered around large language models.

In [15]:
display_summary("https://cnn.com")

# Website Summary: CNN

CNN is a leading news platform providing breaking news, latest headlines, and video content across a range of topics including politics, business, health, and global events. The site covers significant global issues such as the ongoing Ukraine-Russia war and the Israel-Hamas conflict, with live updates and analyses. 

## Recent News Highlights:
- **Israel-Hamas Conflict**: The Israel Defense Forces (IDF) chief has indicated a potential ground incursion in Lebanon amid rising tensions with Hezbollah, with calls from the US and allies for a 21-day ceasefire to avert regional war.
- **Geopolitical Developments**: Israel intercepted a Hezbollah missile aimed at Tel Aviv, marking a significant escalation in hostilities.
- **International Concerns**: There are fears regarding potential Russian attacks on Ukrainian nuclear facilities.
- **Cultural Updates**: A panda pair was welcomed in Hong Kong as China commemorates 75 years of Communist rule.
- **Scientific Discoveries**: Astronomers are reassessing their understanding of the universe following the discovery of massive black hole jets.

CNN also covers various other topics including entertainment, health, science, and climate-related news, showcasing a comprehensive view of both domestic and international affairs.

In [16]:
display_summary("https://anthropic.com")

# Anthropic Website Summary

Anthropic is an AI safety and research company based in San Francisco, dedicated to developing AI models that prioritize safety and reliability. Their key AI model currently available is **Claude 3.5 Sonnet**, which is highlighted as their most intelligent model yet, released on **June 21, 2024**. The company also offers an API that allows businesses to utilize Claude for improved efficiency and new revenue streams.

## Notable Announcements:
- **Claude 3.5 Sonnet Released**: June 21, 2024
- **Research on Harmlessness from AI Feedback**: December 15, 2022
- **Core Views on AI Safety**: March 8, 2023

Anthropic emphasizes an interdisciplinary approach, bringing together expertise from machine learning, physics, policy, and product development to further their mission in safe AI research. They actively seek to expand their team through open positions.

In [19]:
display_summary("https://www.visitsingapore.com")

# Summary of Visit Singapore Official Site

The **Visit Singapore Official Site** serves as a comprehensive guide for tourists and locals eager to explore the myriad attractions that Singapore has to offer. The website features detailed information on various categories including:

- **Top Attractions**: Highlights of popular places to visit, such as Gardens by the Bay, Sentosa Island, and Universal Studios Singapore.
- **Cultural Experiences**: Insights into Singapore's diverse heritage and cultural festivals.
- **Dining Options**: Recommendations for local cuisine, hawker centers, and fine dining establishments.
- **Shopping**: Guides on where to shop, including famous shopping streets and malls.
- **Events and Festivals**: Information on upcoming events and annual festivals that showcase Singapore’s vibrant lifestyle.

The site also emphasizes the city’s safety and cleanliness, making it an appealing destination for travelers.

### News and Announcements
No specific news or announcements were highlighted in the provided content.

In [20]:
display_summary("https://www.gardensbythebay.com.sg/")

# Gardens by the Bay - Summary

Gardens by the Bay is a premier horticultural attraction located in the heart of Singapore, renowned for its diverse collection of over 1.5 million plants from around the world, excluding Antarctica. The site features iconic structures and attractions such as the Flower Dome, Cloud Forest, OCBC Skyway, and Supertree Observatory, creating a unique blend of nature and architecture.

## Highlights
- **Attractions**: Visitors can explore various themed conservatories, interact with art sculptures, and enjoy panoramic views from the Skyway.
- **Events**: Noteworthy upcoming events include the "Carnival of Flowers" running from September 23 to November 17, 2024, and seasonal craft activities in the Flower Dome.
- **Sustainability**: The gardens emphasize sustainability through innovative architecture and eco-friendly practices.

## Promotions and Membership
- Current promotions include a 15% discount on Friends of the Gardens membership for DBS/POSB cardholders until October 31, 2024, and ongoing deals for dining within the attraction.
- A chance to win air tickets to Europe is offered for new Friends of the Gardens members from September 1, 2024, to May 31, 2025.

The website serves as a comprehensive guide for planning visits, offers educational resources for schools, and encourages engagement through social media platforms.